In [1]:
def update_parent_dict(parent_dict, key, new_dict):
    if key in parent_dict:
        # If the key exists, append the new dict to the existing value
        if isinstance(parent_dict[key], list):
            parent_dict[key].append(new_dict)
        else:
            parent_dict[key] = [parent_dict[key], new_dict]
    else:
        # If the key is new, add it to the parent dict
        parent_dict[key] = new_dict

# Example usage
parent = {}
update_parent_dict(parent, 'a', {'name': 'Alice'})
update_parent_dict(parent, 'b', {'name': 'Bob'})
update_parent_dict(parent, 'a', {'name': 'Charlie'})

print(parent)

{'a': [{'name': 'Alice'}, {'name': 'Charlie'}], 'b': {'name': 'Bob'}}


In [7]:
from datetime import datetime, timedelta
import dateutil.relativedelta as REL

def get_last_weekday(date, weekday1):
    if date is None:
        date = datetime.today()

    # Get the day of the week (0 = Monday, 6 = Sunday)
    day_of_week = date.weekday()

    # Calculate the number of days to go back to the previous Monday
    days_to_subtract = (day_of_week + 7 - weekday1) % 7 

    # Subtract the days to get last Monday
    last_monday = date - timedelta(days=days_to_subtract)

    return last_monday

# Example usage
last_monday = get_last_weekday(datetime.today(), 3)
print(last_monday.strftime("%Y-%m-%d"))

2024-09-26


In [15]:
from datetime import datetime
from dateutil.relativedelta import relativedelta, FR, SA, MO

# Assume 'date' is your starting date
date = datetime(2024, 9, 23)  # Example starting date
offset = 1  # Example offset

# Calculate the new date
new_date = date + relativedelta(weekday=MO, days=offset * 7)
print(new_date)

2024-09-30 00:00:00


In [2]:
import ast
l = ast.literal_eval('[7, 8, 10]')
l

[7, 8, 10]

In [3]:
type(l)

list

In [ ]:
{"chart_id": 8, "page_number": 2, "chart_title": "Column Chart 2", "chart_desc": "demo2", "chart_coordinates": {"x_origin": 0, "y_origin": 0, "width_x": 401, "height_y": 250}, "metrics": [{"metrics_object": ["DR:1028609"], "alias": "Report_18132", "metrics_filter": [], "dimensions_slice": null}, {"metrics_object": ["DR:1011390"], "alias": "Report_18102", "metrics_filter": [], "dimensions_slice": null}, {"metrics_object": ["DR:1028849"], "alias": "2222", "metrics_filter": [], "dimensions_slice": null}], "period_range": {"period_start": ["Month", "-0"], "interval": ["Monthly", "-1"], "length": 6}, "chart_type": 1, "switch_ind": true}

In [ ]:
import json
import boto3
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import io

# Initialize S3 client outside of handler to reuse across Lambda invocations
s3_client = boto3.client('s3')

# Function to read a single file from S3
def read_s3_file(bucket_name, key):
    try:
        response = s3_client.get_object(Bucket=bucket_name, Key=key)
        file_content = response['Body'].read().decode('utf-8')
        # Read the CSV content into a DataFrame
        df = pd.read_csv(io.StringIO(file_content))
        return df
    except Exception as e:
        print(f"Error reading {key}: {str(e)}")
        return None

# Lambda handler
def lambda_handler(event, context):
    # List of file keys in your S3 bucket (could be passed in through the event)
    s3_files = [
        'file1.csv',
        'file2.csv',
        'file3.csv'
    ]
    
    bucket_name = 'your-s3-bucket'

    # Use ThreadPoolExecutor to read files in parallel
    with ThreadPoolExecutor(max_workers=5) as executor:
        # Launch parallel S3 read operations
        futures = [executor.submit(read_s3_file, bucket_name, file) for file in s3_files]

        # Collect the results
        dataframes = [future.result() for future in futures if future.result() is not None]

    # Combine DataFrames (if applicable)
    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        print("Combined DataFrame:")
        print(combined_df)

    return {
        'statusCode': 200,
        'body': json.dumps("Files processed successfully!")
    }
